In [ ]:
from datasets import load_dataset

In [ ]:
ds = load_dataset("keremberke/pothole-segmentation", "full")

In [ ]:
train_dataset = ds["train"]

In [ ]:
validation_dataset = ds["validation"]

In [ ]:
test_dataset = ds["test"]

In [ ]:
import os

# Create a directory to store YOLO-formatted labels
output_dir = "yolo_formatted_data"
os.makedirs(output_dir, exist_ok=True)

# Function to convert bounding boxes to YOLO format
def process_json_entry(json_entry):
    image = json_entry["image"]
    img_width, img_height = json_entry["width"], json_entry["height"]
    
    # Retrieve bounding boxes and categories
    bboxes = json_entry["objects"]["bbox"]
    categories = json_entry["objects"]["category"]

    yolo_boxes = []
    for box, class_id in zip(bboxes, categories):
        x_min, y_min, width, height = box
        
        # Convert to YOLO format
        x_center = (x_min + width / 2) / img_width
        y_center = (y_min + height / 2) / img_height
        width /= img_width
        height /= img_height
        
        # Append formatted data
        yolo_boxes.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return image, yolo_boxes

# Process each entry and save the labels
for i, entry in enumerate(train_dataset):
    image, yolo_boxes = process_json_entry(entry)
    
    # Save image to output directory
    image_basename = f"train_image_{i}.jpg"
    image.save(os.path.join(output_dir, image_basename))
    
    # Save bounding box annotations to a .txt file with the same name as the image
    label_path = os.path.join(output_dir, f"{os.path.splitext(image_basename)[0]}.txt")
    with open(label_path, "w") as f:
        f.write("\n".join(yolo_boxes))

print("YOLO formatted data saved successfully.")

In [ ]:
output_dir = "validation_formatted_data"
os.makedirs(output_dir, exist_ok=True) 
for i, entry in enumerate(validation_dataset):
    image, yolo_boxes = process_json_entry(entry)
    
    # Save image to output directory
    image_basename = f"train_image_{i}.jpg"
    image.save(os.path.join(output_dir, image_basename))
    
    # Save bounding box annotations to a .txt file with the same name as the image
    label_path = os.path.join(output_dir, f"{os.path.splitext(image_basename)[0]}.txt")
    with open(label_path, "w") as f:
        f.write("\n".join(yolo_boxes))

print("YOLO formatted data saved successfully.")

In [1]:
import torch

# Save original torch.load
orig_torch_load = torch.load

def patched_load(f, *args, **kwargs):
    # Force weights_only=False
    kwargs['weights_only'] = False
    return orig_torch_load(f, *args, **kwargs)

# Patch it
torch.load = patched_load

# Now import YOLO
from ultralytics import YOLO

# Try loading
model = YOLO("yolov8n.pt")

In [ ]:
# Train the model
model.train(
    data="C:\\Users\\Aaryan Naithani\\OneDrive\\Desktop\\Pothole_Detection_Model_Using_YOLOv8-main\\pothole_dataset.yaml",  # Path to your dataset YAML file
    epochs=1800,  # Number of epochs (adjust as needed)
    batch=4,  # Batch size
    imgsz=640,  # Image size (YOLOv8 default)
    name="pothole_detector",  # Model name for saving results
    device="cuda",
    resume="True"
)

In [3]:
metrics = model.val()
print(metrics) 

In [ ]:
import os

# Define output directories
output_dir = "test_yolo_formatted_data"
images_dir = os.path.join(output_dir, "images")
labels_dir = os.path.join(output_dir, "labels")
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Function to convert bounding boxes to YOLO format
def process_json_entry(json_entry):
    image = json_entry["image"]
    img_width, img_height = json_entry["width"], json_entry["height"]
    
    # Retrieve bounding boxes and categories
    bboxes = json_entry["objects"]["bbox"]
    categories = json_entry["objects"]["category"]

    yolo_boxes = []
    for box, class_id in zip(bboxes, categories):
        x_min, y_min, width, height = box
        
        # Convert to YOLO format
        x_center = (x_min + width / 2) / img_width
        y_center = (y_min + height / 2) / img_height
        width /= img_width
        height /= img_height
        
        # Append formatted data
        yolo_boxes.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return image, yolo_boxes

# Process each entry in the test dataset and save
for i, entry in enumerate(test_dataset):
    # Process image and YOLO boxes from JSON entry
    image, yolo_boxes = process_json_entry(entry)
    
    # Save image to output directory
    image_basename = f"test_image_{entry['image_id']}.jpg"
    image.save(os.path.join(images_dir, image_basename))
    
    # Save bounding box annotations to a .txt file with the same name as the image
    label_path = os.path.join(labels_dir, f"{os.path.splitext(image_basename)[0]}.txt")
    with open(label_path, "w") as f:
        f.write("\n".join(yolo_boxes))

print("Test YOLO formatted data saved successfully.")


In [ ]:
from ultralytics import YOLO
import os

# Load your trained YOLOv8 model
model = YOLO("C:\\Users\\Aaryan Naithani\\OneDrive\\Desktop\\Pothole_Detection_Model_Using_YOLOv8-main\\runs\\detect\\pothole_detector6\\weights\\best.pt")
 # Replace with the path to your trained model weights

# Define the path to your test images
test_images_dir = "C:\\Users\\Aaryan Naithani\\OneDrive\\Desktop\\Pothole_Detection_Model_Using_YOLOv8-main\\yolo_formatted_data\\images\\test"

# Run predictions on the test images
results = model.predict(source=test_images_dir, save=True, save_txt=True)  # save=True saves annotated images; save_txt=True saves predictions in txt format

print("Test predictions completed and saved.")